# Detect the Grizlly Bear

In [ ]:
!pip install fastai -Uqq fastai

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(keywords, max_results = 100):
    "Search for images using DuckDuckGo and return the URLs of the images."
    return L(DDGS().images(keywords, max_results = max_results)).itemgot("image")

In [ ]:
#Search from grizzly photo
urls = search_images("grizzly bear", max_results =1)

urls[0]

In [ ]:
from fastdownload import download_url
dest = "grizzly bear"
download_url(urls[0], dest, show_progress = True)


from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256, 256)

In [ ]:
#Download images
bear_types = 'grizzly', 'black', 'polar'
path = Path("bears")

if not path.exists():
    path.mkdir()
    for bear in bear_types:
        dest = (path/bear)
        dest.mkdir(exist_ok = True)
        results = search_images(f"{bear} bear")
        download_images(dest, urls = results)

In [ ]:
#Delete failed images
fns = get_image_files(path)

failed = verify_images(fns)
len(failed)

In [ ]:
failed.map(Path.unlink)

In [ ]:
from fastai.vision.all import *

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128),
    batch_tfms=aug_transforms(mult=1.0)
)

In [ ]:
dls = bears.dataloaders(path, bs=4)
dls.show_batch(max_n=9)  # Test display

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
!pip install ipywidgets -Uqq

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
learn.export()